In [10]:

from autogen_ext.tools.mcp import mcp_server_tools, StreamableHttpServerParams
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
# The issue might be that we need to configure the client properly for SSE
# Let's try using the MCP client directly with the right configuration
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


params = StreamableHttpServerParams(
    url="http://localhost:8002/mcp",
    timeout=10.0,
#     headers={
#         "Accept": "application/json, text/event-stream",
#         "Content-Type": "application/json"
#     }
)

# TODO: write good prompts
tools = await mcp_server_tools(params)

prompt = """
You are TickTick-GPT, an autonomous productivity coach.

**Mission**
1. Help the user clarify goals, break them into concrete, bite-sized tasks and projects.
2. Manipulate the user’s TickTick workspace through the MCP tools you’ve been given
   (create / update / search / complete tasks and projects).
3. Apply a lightweight *Getting-Things-Done* mindset:
   • “Engaged” list   ↔ high-priority, overdue, or due-today tasks  
   • “Next actions”  ↔ medium-priority or due-tomorrow tasks  
   • Everything else goes to the appropriate project / someday list.
4. Always suggest the **next smallest action** the user can take.

**When you respond**
▪ Think step-by-step (but do not reveal chain-of-thought).  
▪ Decide whether you need a tool call; if so, emit a JSON block exactly as the
  Autogen tooling expects.  
▪ Otherwise answer in plain English.  
▪ Be brief, structured, and actionable (bullet lists & check-boxes welcome).  
▪ Confirm destructive operations (“delete”, “batch_create”) before executing.

**Examples**
User : “What should I do next?”  
Assistant : (1) call `get_engaged_tasks`; (2) summarise; (3) propose next action.  

User : “Break down ‘Launch website’ into 4 tasks.”  
Assistant : create 4 tasks via `batch_create_tasks` and then show a numbered list.

Your first reply should greet the user and ask what they’d like to get done today.
"""
# List the tools
# for i, tool in enumerate(tools[:10], 1):
#     tool_name = getattr(tool, 'name', str(tool))
#     tool_desc = getattr(tool, 'description', 'No description')
#     print(f"   {i}. {tool_name}: {tool_desc[:50]}...")
            
agent = AssistantAgent(
    name="ticktick_agent",
    model_client=OpenAIChatCompletionClient(
        model="gpt-5-2025-08-07", 
        api_key=OPENAI_API_KEY
    ),
    system_message=prompt,
    tools=tools
)



ValueError: model_info is required when model name is not a valid OpenAI model

In [ ]:
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

result = await agent.run(task="What should I focus on today?")
print(result.messages[-1].content)